In [25]:
# Bukola Grace Omotoso
# CSCI 6350-001
# Project 3
# Due Date: 02/14/2020

# Naive Bayes and Sentiment Classification



In [26]:
import random;
from nltk.classify import NaiveBayesClassifier;
from nltk import classify;
from nltk.metrics import *;
import collections;
import re
from collections import Counter 
import string


In [27]:
#   import state of the union speech
def importSotu():
	infile = open("sotu.txt", "r");
	sotu_file = infile.readlines();
	sotu = [];
	for line in sotu_file:
		if not line.strip():
			continue;
		else:
			sotu.append(line.strip()+" ");
	return sotu;

In [28]:
#   create data structures for each party's speeches
def separateSotuByParties(sotu, republicans, democrats):
        all_presidents = [];
        republican_sotu = [];
        democrat_sotu = [];
        independent_sotu = [];
        dem = False;
        rep = False;
        ind = False;
        speeches = "";
        for line in sotu:
                if line[0:3] =="***": # A president's name, not part of speech
                        #print(line);
                        if rep == True and speeches != "":
                                republican_sotu.append(speeches);
                                speeches = "";
                        elif dem == True and speeches != "":
                                democrat_sotu.append(speeches);
                                speeches ="";
                        else:
                                if speeches != "":
                                        independent_sotu.append(speeches);
                                        speeches = "";
                        i1 = line.find("@");
                        i2 = line.find("@", i1+1);
                        president = line[i1+1:i2];
                        #print(president);
                        #if president not in all_presidents:
                         #   all_presidents.append(president);
                        if president in democrats: 
                                dem = True;
                                rep = False;
                                ind = False;
                                continue;
                        elif president in republicans:
                                rep = True;
                                dem = False;
                                ind = False;
                                continue;
                        else:
                                ind = True;
                                rep = False;
                                dem = False;
                                continue;
                else:
                        speeches+=line;
                        
        if rep == True and speeches != "":
                republican_sotu.append(speeches);
        elif dem == True and speeches != "":
                democrat_sotu.append(speeches);
                        
        #print(all_presidents);
        #print(republican_sotu);
        
        return(republican_sotu, democrat_sotu);

In [29]:
def getDemocratRepublicanSotu():    
    # Create a list of republican presidents
    republicans = ['Abraham Lincoln', 'Ulysses S. Grant', 'Rutherford B. Hayes', 'Chester A. Arthur',  
                       'Benjamin Harrison',  'William McKinley', 'Theodore Roosevelt', 'William H. Taft', 
                       'Warren Harding', 'Calvin Coolidge', 'Herbert Hoover',  'Dwight D. Eisenhower',
                       'Richard Nixon', 'Gerald R. Ford', 'Ronald Reagan', 'George H.W. Bush', 'George W. Bush', 
                       'Donald Trump' ];

    # Create a list of democratic presidents
    democrats = ['Andrew Jackson', 'Martin van Buren','Martin Van Buren','James Polk', 'Franklin Pierce',  'James Buchanan',  
                 'Grover Cleveland', 'Woodrow Wilson' , 'Franklin D. Roosevelt',  'Harry S. Truman',  'Harry Truman',
                 'John F. Kennedy',  'Lydon Johnson', 'Lyndon B. Johnson',  'Jimmy Carter', 'Bill Clinton', 'William J. Clinton', 'Barack Obama']

    # import the state of the union speech
    sotu = importSotu();

    # Split the state of the union speech by party
    republican_sotu, democrat_sotu = separateSotuByParties(sotu, republicans, democrats);

    # Merge both speeeches to get all the speech
    #Shuffle each parties' sotu (to guarantee randomness) before merging 
    random.shuffle(republican_sotu);
    random.shuffle(democrat_sotu);

    all_sotu = republican_sotu + democrat_sotu;

    # Shuffle all the speeches again to guarantee randomness
    random.shuffle(all_sotu);

    #build a single list consisting of tuples made up of a state of the union speech
    #and the party of the president who gave the speech
    labeled_sotu = ([(speech, 'republican') for speech in republican_sotu] + [(speech, 'democrat') for speech in democrat_sotu]);
    return labeled_sotu;

In [30]:
# feature function - accepts a string(speech) and returns a dictionary with key
# of some common features in opening and closin sentences in the speech

def sentence_features(speech):
    #get the opening sentence from the speech
    openingsentence = speech.split(':')[0];
    closingsentence = speech.split('.')[-2];

    
    # Words common to a democrat president in an opening speech
    commonDemOpeningword = ['fellow',  'madam speaker', 'mr. speaker', 'vice president' , 'my fellow Americans', 'the united states', 'fellow citizens'];
    hasCommonDemOpening = False;
    
    #if the opening sentence has the word Senate and House of Representatives,
    #it is probably a speech by a republican president
    commonRepOpeningWord = ['Senate and House of Representatives']
    hasCommonRepOpening = False;
    
    #if the opening sentence has the word Fellow, it is probably a speech by a democrat president
    for word in commonDemOpeningword:
        if (word in openingsentence):
            hasCommonDemOpening = True;
            
    for word in commonRepOpeningWord:
        if (word in openingsentence):
            hasCommonRepOpening = True;
    
    #if the closing sentence has the word America, it is probably said by a republican president
    commonRepClosingWord = ['broadcast live' ]
    hascommonRepClosingWord = False;

     #if the closing sentence has the word America, it is probably said by a republican president
    commonDemClosingWord = ['thank you', ]
    hascommonDemClosingWord = False;

    for word in commonDemClosingWord:
        if (word in closingsentence):
            hascommonDemClosingWord = True;
            
    for word in commonRepClosingWord:
        if (word in closingsentence):
            hascommonRepClosingWord = True;
    


    return {'hasCommonDemOpening': hasCommonDemOpening, 'hasCommonRepOpening': hasCommonRepOpening,
            'hascommonDemClosingWord':hascommonDemClosingWord, 'hascommonRepClosingWord':hascommonRepClosingWord};


In [31]:
def main():
    labeled_sotu = getDemocratRepublicanSotu();
    random.shuffle(labeled_sotu);
    
    # construct a list consisting of the opening sentence feature dictionary: The opening sentence in a speech
    # and the party of the president who made the speech
    featuresets = [(sentence_features(speech), party) for (speech, party) in labeled_sotu];
    #print (featuresets);
    
    # Split data into training and test
    cut = int(181 * 0.20);
    train_set, test_set = featuresets[cut:], featuresets[:cut];
    
    #using the training set to train the classifer,
    classifier = NaiveBayesClassifier.train(train_set)
    # classify each entry in the test set individually
    # and capture precision, recall, and F1\n",
    refsets = collections.defaultdict(set);
    testsets = collections.defaultdict(set);
    for i, (features, sotu_label) in enumerate(test_set):
        #print(i, (features, sotu_label));
        refsets[sotu_label].add(i);
        observed = classifier.classify(features);
        testsets[observed].add(i);

    #initialize variables to keep metrics
    overall_accuracy = 0.0;
    
    dem_precision = 0.0;
    dem_recall = 0.0;
    dem_f1measure = 0.0;
    
    rep_precision = 0.0;
    rep_recall = 0.0;
    rep_f1measure = 0.0;
    
    for i in range(30):
        overall_accuracy += classify.accuracy(classifier, test_set);
        
        dem_precision += precision(refsets['democrat'], testsets['democrat']);
        dem_recall += recall(refsets['democrat'], testsets['democrat']);
        dem_f1measure+= f_measure(refsets['democrat'], testsets['democrat']);
        
        rep_precision += precision(refsets['republican'], testsets['republican']);
        rep_recall += recall(refsets['republican'], testsets['republican']);
        rep_f1measure += f_measure(refsets['democrat'], testsets['democrat']);
    
    # Do 30 runs and print the average
    # Print the accuracy for the overall classifier accuracy
    print("{:40s} {:.2%} ".format("Overall Accuracy: ",(overall_accuracy/30)));

    print();
    #Print statistics for democrat president  speech
    print ("{:40s} {:.2%} ".format("democrat president speech precision:", (dem_precision/30)));
    print ("{:40s} {:.2%} ".format("democrat president speech recall:",(dem_recall/30)));
    print ("{:40s} {:.2%} ".format("democrat president speech F-measure:",(dem_f1measure/30)))

    print();
    #Print statistics for republican president  speech
    print ("{:40s} {:.2%} ".format("republican president speech precision:", (rep_precision/30)));
    print ("{:40s} {:.2%} ".format("republican president speech recall:", (rep_recall/30)));
    print ("{:40s} {:.2%} ".format("republican president speech F-measure:",(rep_f1measure/30)))

In [32]:
main();

Overall Accuracy:                        66.67% 

democrat president speech precision:     60.00% 
democrat president speech recall:        88.24% 
democrat president speech F-measure:     71.43% 

republican president speech precision:   81.82% 
republican president speech recall:      47.37% 
republican president speech F-measure:   71.43% 
